In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, auc
from xgboost import XGBClassifier

# Load dataset
df = pd.read_csv("Train.csv")  # Replace with your actual dataset

# Split target and features
y = df.iloc[:, 0]   # First column is the target variable
X = df.iloc[:, 1:]  # Remaining columns are features

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train Isolation Forest
iso_forest = IsolationForest(n_estimators=200, contamination=0.028, random_state=42)
iso_forest.fit(X_train)

# Generate Anomaly Scores as a New Feature
X_train["anomaly_score"] = iso_forest.decision_function(X_train)
X_test["anomaly_score"] = iso_forest.decision_function(X_test)

# Train XGBoost Model
xgb_model = XGBClassifier(scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
                          eval_metric="logloss", use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# Predict Probabilities
y_pred_prob = xgb_model.predict_proba(X_test)[:, 1]



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:09:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [2]:
# Adjust Decision Threshold
threshold = 0.4  # Tune this value based on PR curve
y_pred = (y_pred_prob >= threshold).astype(int)

# Evaluation
print(classification_report(y_test, y_pred))

# Precision-Recall AUC
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall, precision)
print(f"Precision-Recall AUC: {pr_auc:.4f}")


              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1060
           1       0.58      0.48      0.53        31

    accuracy                           0.98      1091
   macro avg       0.78      0.74      0.76      1091
weighted avg       0.97      0.98      0.97      1091

Precision-Recall AUC: 0.5189
